In [7]:
import json
import os
import glob
from PIL import Image

# ===== 경로 설정 =====
label_folder = r"C:\Users\USER\Downloads\korean_emotion_complex_vision_1_percent\labels"
image_folder = r"C:\Users\USER\Downloads\korean_emotion_complex_vision_1_percent"

# ===== 이미지 정보 함수 =====
def get_image_info(img_path):
    try:
        with Image.open(img_path) as img:
            width, height = img.size
            mode = img.mode
        file_size_kb = os.path.getsize(img_path) / 1024
        return width, height, mode, file_size_kb
    except:
        return None, None, None, None

# ===== 1. JSON 여러 개 읽기 =====
all_entries = []

for json_file in glob.glob(os.path.join(label_folder, "*.json")):
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    if isinstance(data, dict):
        for fname, info in data.items():
            entry = {"file_name": fname}
            if isinstance(info, dict):
                entry.update(info)
            all_entries.append(entry)

    elif isinstance(data, list):
        for item in data:
            # file_name 없으면 건너뛰기
            if "file_name" not in item:
                print(f"⚠️ file_name 없어서 건너뜀: {item}")
                continue
            all_entries.append(item)

print(f"📌 전체 라벨 수: {len(all_entries)}")

# ===== 2. 이미지 전체 경로 매핑 =====
image_paths = glob.glob(os.path.join(image_folder, "**", "*.jpg"), recursive=True)
image_paths += glob.glob(os.path.join(image_folder, "**", "*.png"), recursive=True)
image_map = {os.path.basename(p): p for p in image_paths}

print(f"📌 전체 이미지 수: {len(image_map)}")

# ===== 3. 샘플 5개 이미지 정보 출력 =====
print("\n📌 이미지 기본 정보 (샘플 5개):")
count = 0
for entry in all_entries:
    if "file_name" not in entry:
        continue
    img_name = os.path.basename(entry["file_name"])
    if img_name in image_map:
        img_path = image_map[img_name]
        w, h, mode, size_kb = get_image_info(img_path)
        print(f"{img_name} → {w}x{h}px, 모드:{mode}, 크기:{size_kb:.1f}KB, 라벨:{entry.get('emotion')}")
    else:
        print(f"{img_name} → ❌ 이미지 없음")
    count += 1
    if count >= 5:
        break

⚠️ file_name 없어서 건너뜀: {'filename': '5a30dc193e2c6144184968043c00773270b111a23a3c83b00f3971c9cc8a670d_남_30_기쁨_도심 환경_20201205004857-008-002.jpg', 'gender': '남', 'age': 30, 'isProf': '전문인', 'faceExp_uploader': '기쁨', 'bg_uploader': '도심 환경', 'annot_A': {'boxes': {'maxX': 1905.6854, 'maxY': 1463.4427, 'minX': 1110.94, 'minY': 440.67734}, 'faceExp': '기쁨', 'bg': '도심 환경'}, 'annot_B': {'boxes': {'maxX': 1933.95158397587, 'maxY': 1513.78521741175, 'minX': 1013.45109777892, 'minY': 571.844123447761}, 'faceExp': '기쁨', 'bg': '도심 환경'}, 'annot_C': {'boxes': {'maxX': 1884.30234826051, 'maxY': 1506.37633552483, 'minX': 1122.77513130549, 'minY': 459.83909354858}, 'faceExp': '기쁨', 'bg': '도심 환경'}}
⚠️ file_name 없어서 건너뜀: {'filename': '0d18ae1ed7d9e912864b736523a94a24db0595bf273964cb532e62a6087cf325_여_30_기쁨_행사&사무공간_20201207000342-006-001.jpg', 'gender': '여', 'age': 30, 'isProf': '전문인', 'faceExp_uploader': '기쁨', 'bg_uploader': '행사/사무공간', 'annot_A': {'boxes': {'maxX': 1851.23903563087, 'maxY': 1499.8602989002, 

In [17]:
import pandas as pd
import json

# 7가지 감정 파일 이름 목록
emotions = ['기쁨', '슬픔', '분노', '불안', '당황', '상처', '중립']
all_data = []

for emotion in emotions:
    file_path = f'{emotion}_sampled.json'
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 각 JSON 파일의 데이터를 처리
    for item in data:
        filename = item['filename']
        uploader_info = {
            'gender': item['gender'],
            'age': item['age'],
            'isProf': item['isProf'],
            'faceExp_uploader': item['faceExp_uploader'],
            'bg_uploader': item['bg_uploader']
        }

        # 3명의 어노테이터 데이터 추출 및 리스트에 추가
        for annot_key in ['annot_A', 'annot_B', 'annot_C']:
            annot_data = item.get(annot_key, {}) # 데이터가 없는 경우를 대비
            if annot_data:
                annot_info = {
                    'filename': filename,
                    'annotator': annot_key,
                    'faceExp_annotator': annot_data.get('faceExp'),
                    'bg_annotator': annot_data.get('bg'),
                    'minX': annot_data.get('boxes', {}).get('minX'),
                    'minY': annot_data.get('boxes', {}).get('minY'),
                    'maxX': annot_data.get('boxes', {}).get('maxX'),
                    'maxY': annot_data.get('boxes', {}).get('maxY')
                }
                annot_info.update(uploader_info)
                all_data.append(annot_info)

# 통합된 데이터를 DataFrame으로 변환
df = pd.DataFrame(all_data)

print("통합된 데이터프레임의 상위 5개 행:")
print(df.head())
print("\n데이터프레임 정보:")
df.info()

ModuleNotFoundError: No module named 'pandas'

In [18]:
import json
import os
import glob
from PIL import Image

# ===== 경로 설정 =====
label_folder = r"C:\Users\USER\Downloads\korean_emotion_complex_vision_1_percent\labels"   # JSON들이 들어있는 폴더
image_folder = r"C:\Users\USER\Downloads\korean_emotion_complex_vision_1_percent"          # 이미지들이 들어있는 상위 폴더

# ===== 이미지 정보 함수 =====
def get_image_info(img_path):
    try:
        with Image.open(img_path) as img:
            width, height = img.size
            mode = img.mode
        file_size_kb = os.path.getsize(img_path) / 1024
        return width, height, mode, file_size_kb
    except:
        return None, None, None, None

# ===== 1. JSON 여러 개 읽기 =====
all_entries = []

for json_file in glob.glob(os.path.join(label_folder, "*.json")):
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    if isinstance(data, dict):
        for fname, info in data.items():
            all_entries.append({"file_name": fname, **info})
    elif isinstance(data, list):
        all_entries.extend(data)

print(f"📌 전체 라벨 수: {len(all_entries)}")

# ===== 2. 이미지 전체 경로 매핑 =====
image_paths = glob.glob(os.path.join(image_folder, "**", "*.jpg"), recursive=True)
image_paths += glob.glob(os.path.join(image_folder, "**", "*.png"), recursive=True)
image_map = {os.path.basename(p): p for p in image_paths}

print(f"📌 전체 이미지 수: {len(image_map)}")

# ===== 3. 샘플 5개 이미지 정보 출력 =====
print("\n📌 이미지 기본 정보 (샘플 5개):")
for entry in all_entries[:5]:
    img_name = os.path.basename(entry["file_name"])
    if img_name in image_map:
        img_path = image_map[img_name]
        w, h, mode, size_kb = get_image_info(img_path)
        print(f"{img_name} → {w}x{h}px, 모드:{mode}, 크기:{size_kb:.1f}KB, 라벨:{entry.get('emotion')}")
    else:
        print(f"{img_name} → ❌ 이미지 없음")

📌 전체 라벨 수: 2210
📌 전체 이미지 수: 2161

📌 이미지 기본 정보 (샘플 5개):


KeyError: 'file_name'

In [20]:
import json
import os
import glob
from PIL import Image

# ===== 경로 설정 =====
label_folder = r"C:\Users\USER\Downloads\korean_emotion_complex_vision_1_percent\labels"
image_folder = r"C:\Users\USER\Downloads\korean_emotion_complex_vision_1_percent"

# ===== 이미지 정보 함수 =====
def get_image_info(img_path):
    try:
        with Image.open(img_path) as img:
            width, height = img.size
            mode = img.mode
        file_size_kb = os.path.getsize(img_path) / 1024
        return width, height, mode, file_size_kb
    except:
        return None, None, None, None

# ===== 1. JSON 여러 개 읽기 =====
all_entries = []

for json_file in glob.glob(os.path.join(label_folder, "*.json")):
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    if isinstance(data, dict):
        for fname, info in data.items():
            entry = {"file_name": fname}
            if isinstance(info, dict):
                entry.update(info)
            all_entries.append(entry)

    elif isinstance(data, list):
        for item in data:
            # file_name 없으면 건너뛰기
            if "file_name" not in item and "filename" not in item:
                print(f"⚠️ file_name 또는 filename 없어서 건너뜀: {item}")
                continue

# file_name 키가 없고 filename이 있으면 filename을 file_name으로 바꿔줌
if "file_name" not in item and "filename" in item:
    item["file_name"] = item.pop("filename")

all_entries.append(item)

print(f"📌 전체 라벨 수: {len(all_entries)}")

# ===== 2. 이미지 전체 경로 매핑 =====
image_paths = glob.glob(os.path.join(image_folder, "**", "*.jpg"), recursive=True)
image_paths += glob.glob(os.path.join(image_folder, "**", "*.png"), recursive=True)
image_map = {os.path.basename(p): p for p in image_paths}

print(f"📌 전체 이미지 수: {len(image_map)}")

# ===== 3. 샘플 5개 이미지 정보 출력 =====
print("\n📌 이미지 기본 정보 (샘플 5개):")
count = 0
for entry in all_entries:
    if "file_name" not in entry:
        continue
    img_name = os.path.basename(entry["file_name"])
    if img_name in image_map:
        img_path = image_map[img_name]
        w, h, mode, size_kb = get_image_info(img_path)
        print(f"{img_name} → {w}x{h}px, 모드:{mode}, 크기:{size_kb:.1f}KB, 라벨:{entry.get('emotion')}")
    else:
        print(f"{img_name} → ❌ 이미지 없음")
    count += 1
    if count >= 5:
        break

📌 전체 라벨 수: 1
📌 전체 이미지 수: 2161

📌 이미지 기본 정보 (샘플 5개):
4551a4c165093932b583c335b2c33abf357677f74c736c5e0ee8eb3b6c1306df_남_20_중립_교통&이동수단(엘리베이터 포함)_20210203064537-002-014.jpg → 3264x1836px, 모드:RGB, 크기:1172.0KB, 라벨:None


In [15]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [10]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   -------- ------------------------------- 2.4/11.0 MB 11.7 MB/s eta 0:00:01
   ------------------ --------------------- 5.0/11.0 MB 11.6 MB/s eta 0:00:01
   -------------------------- ------------- 7.3/11.0 MB 11.6 MB/s eta 0:00:01
   --------------------------------- ------ 9.2/11.0 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 10.4 MB/s eta 0:00:00
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

   ---------------------------------------- 0/3 [pytz]
   ---------------------------------------- 0/3 [pytz]
   ---------------------------------------- 0/3 [pytz]
   ---------------------------------------- 0/3 [p

In [21]:
import os
import json
from collections import Counter

# ====== 경로 설정 ======
json_folder = r"C:\Users\USER\Downloads\korean_emotion_complex_vision_1_percent\labels"
image_folder = r"C:\Users\USER\Downloads\korean_emotion_complex_vision_1_percent"

# ====== JSON 읽기 ======
all_entries = []
for fname in os.listdir(json_folder):
    if fname.lower().endswith(".json"):
        fpath = os.path.join(json_folder, fname)
        with open(fpath, "r", encoding="utf-8") as f:
            data = json.load(f)

        if isinstance(data, dict):
            for k, v in data.items():
                all_entries.append({
                    "file_name": k,
                    "emotion": v.get("emotion"),
                    "bbox": v.get("bbox")
                })
        elif isinstance(data, list):
            for item in data:
                all_entries.append({
                    "file_name": item.get("file_name"),
                    "emotion": item.get("emotion"),
                    "bbox": item.get("bbox")
                })

print(f"📌 총 라벨 수: {len(all_entries)}")

# ====== 이미지 폴더 읽기 ======
valid_exts = {".jpg", ".jpeg", ".png"}
image_files = []
for root, _, files in os.walk(image_folder):
    for file in files:
        if os.path.splitext(file)[1].lower() in valid_exts:
            image_files.append(file)

print(f"📌 총 이미지 수: {len(image_files)}")

# ====== 매칭 확인 ======
json_image_set = {os.path.basename(e["file_name"]) for e in all_entries if e["file_name"]}
folder_image_set = set(image_files)

only_in_json = json_image_set - folder_image_set
only_in_folder = folder_image_set - json_image_set
in_both = json_image_set & folder_image_set

print(f"✅ JSON에만 있는 이미지: {len(only_in_json)}")
print(f"✅ 폴더에만 있는 이미지: {len(only_in_folder)}")
print(f"✅ 둘 다 있는 이미지: {len(in_both)}")

# ====== 감정별 개수 ======
emotion_counts = Counter(e["emotion"] for e in all_entries if e["emotion"])
print("\n📊 감정별 이미지 개수:")
for emotion, cnt in emotion_counts.most_common():
    print(f"{emotion}: {cnt}")

# ====== bbox 개수 분포 ======
bbox_counts = []
for e in all_entries:
    if e["bbox"] is None:
        bbox_counts.append(0)
    elif isinstance(e["bbox"], list) and isinstance(e["bbox"][0], (int, float)):
        bbox_counts.append(1)  # 단일 bbox
    elif isinstance(e["bbox"], list):
        bbox_counts.append(len(e["bbox"]))  # bbox 여러 개
    else:
        bbox_counts.append(0)

bbox_count_dist = Counter(bbox_counts)
print("\n📦 bbox 개수 분포:")
for k, v in sorted(bbox_count_dist.items()):
    print(f"{k}개 bbox: {v}개 이미지")

📌 총 라벨 수: 2210
📌 총 이미지 수: 2210
✅ JSON에만 있는 이미지: 0
✅ 폴더에만 있는 이미지: 2210
✅ 둘 다 있는 이미지: 0

📊 감정별 이미지 개수:

📦 bbox 개수 분포:
0개 bbox: 2210개 이미지


In [22]:
import os, glob, json
import unicodedata
from collections import Counter
from difflib import SequenceMatcher

# 경로 설정 (필요시 변경)
label_folder = r"C:\Users\USER\Downloads\korean_emotion_complex_vision_1_percent\labels"
image_folder = r"C:\Users\USER\Downloads\korean_emotion_complex_vision_1_percent"

def normalize_name(s):
    if not s:
        return ""
    s = str(s).strip()
    # 윈도우/URL 경로 제거
    s = s.replace("\\", "/")
    s = os.path.basename(s)
    # 정규화, 소문자화
    s = unicodedata.normalize("NFKC", s).lower()
    return s

# 1) JSON 파일들 로드 + 가능한 filename/emotion/bbox 키 탐색
all_entries = []
key_counter = Counter()
filename_key_counter = Counter()
emotion_key_counter = Counter()
bbox_key_counter = Counter()

json_files = sorted(glob.glob(os.path.join(label_folder, "*.json")))
print("🔎 JSON 파일 개수:", len(json_files))

for jf in json_files[:10]:  # 우선 최대 10개 json만 미리 로드해 구조 파악 (많으면 느릴 수 있음)
    with open(jf, "r", encoding="utf-8") as f:
        data = json.load(f)
    # 구조별로 sample 추출
    if isinstance(data, dict):
        # top-level이 dict인 경우: keys가 파일명일 수 있음
        for k, v in list(data.items())[:50]:  # 각 json에서 최대 50개만 샘플
            entry = {"_top_key": k, "_value_type": type(v).__name__}
            if isinstance(v, dict):
                entry.update(v)
                for kk in v.keys():
                    key_counter[kk]+=1
                # filename-like 키 탐색
                for fk in ("file_name","filename","file","image","img","image_id","name"):
                    if fk in v:
                        filename_key_counter[fk]+=1
                for ek in ("emotion","label","class","faceExp","faceExp_uploader","annot_faceExp"):
                    if ek in v:
                        emotion_key_counter[ek]+=1
                for bk in ("bbox","bboxes","boxes","annot","annotations","box"):
                    if bk in v:
                        bbox_key_counter[bk]+=1
            all_entries.append(entry)
    elif isinstance(data, list):
        for item in data[:200]:  # list이면 처음 200개만
            entry = {}
            if isinstance(item, dict):
                entry.update(item)
                for kk in item.keys():
                    key_counter[kk]+=1
                for fk in ("file_name","filename","file","image","img","image_id","name"):
                    if fk in item:
                        filename_key_counter[fk]+=1
                for ek in ("emotion","label","class","faceExp","faceExp_uploader"):
                    if ek in item:
                        emotion_key_counter[ek]+=1
                for bk in ("bbox","bboxes","boxes","annot","annotations","box"):
                    if bk in item:
                        bbox_key_counter[bk]+=1
            else:
                entry["_val"] = item
            all_entries.append(entry)
    else:
        # 다른 타입
        pass

print("\n📋 JSON 내부에서 관찰된 필드(상위 몇개):")
for k,c in key_counter.most_common(30):
    print(f"  {k}: {c}")

print("\n🔑 filename-like 키 빈도:")
for k,c in filename_key_counter.most_common():
    print(f"  {k}: {c}")

print("\n😊 emotion-like 키 빈도:")
for k,c in emotion_key_counter.most_common():
    print(f"  {k}: {c}")

print("\n📦 bbox-like 키 빈도:")
for k,c in bbox_key_counter.most_common():
    print(f"  {k}: {c}")

# 2) all_entries에서 실제로 파일명(가능한 후보) 리스트 추출 (여러 키 후보 체크)
json_name_candidates = []
for e in all_entries:
    # 우선 많이 쓰이는 키들 순서대로 대입
    name = None
    for fk in ("file_name","filename","file","image","img","image_id","name","_top_key"):
        if fk in e and e[fk]:
            name = e[fk]
            break
    if name:
        json_name_candidates.append(name)

json_name_candidates = [str(x) for x in json_name_candidates]
print("\n샘플 JSON에서 추출한 파일명 후보 (최대 20):")
for s in json_name_candidates[:20]:
    print("  ", repr(s)[:200])

print("\n총 JSON에서 수집된 filename 후보 개수:", len(json_name_candidates))

# 3) 폴더에서 이미지 파일 리스트 수집 (하위폴더 포함)
valid_exts = {".jpg",".jpeg",".png",".bmp",".webp"}
image_paths = []
for root,_,files in os.walk(image_folder):
    for f in files:
        if os.path.splitext(f)[1].lower() in valid_exts:
            image_paths.append(os.path.join(root,f))
image_basenames = [os.path.basename(p) for p in image_paths]

print("🔎 폴더에서 찾은 전체 이미지 개수:", len(image_paths))
print("샘플 이미지 파일명 (최대 20):")
for p in image_basenames[:20]:
    print("  ", p)

# 4) 정규화(소문자+basename) 후 교집합 검사
json_norm = [normalize_name(x) for x in json_name_candidates if x]
img_norm = [normalize_name(x) for x in image_basenames if x]

set_json = set(json_norm)
set_img = set(img_norm)

print("\n교집합 검사:")
print("  JSON 고유 이름 개수:", len(set_json))
print("  IMAGE 고유 이름 개수:", len(set_img))
print("  교집합 개수:", len(set_json & set_img))

# 5) 만약 교집합 0이면 추가 진단: 확장자 제거, 접두사 매칭, prefix 토큰 매칭
intersection = set_json & set_img
if len(intersection) == 0:
    print("\n>>> 교집합이 0입니다. 추가 진단을 수행합니다.")
    # 확장자 제거 비교
    json_noext = {os.path.splitext(n)[0] for n in json_norm}
    img_noext = {os.path.splitext(n)[0] for n in img_norm}
    print("  (확장자 제거) 교집합 개수:", len(json_noext & img_noext))

    # 접두어(언더스코어 앞) 매칭
    def prefix(s): return s.split("_")[0] if s else ""
    json_prefixes = {prefix(n) for n in json_norm}
    img_prefixes = {prefix(n) for n in img_norm}
    common_prefixes = json_prefixes & img_prefixes
    print("  (prefix before '_') JSON prefix 개수:", len(json_prefixes))
    print("  (prefix before '_') IMG prefix 개수:", len(img_prefixes))
    print("  (prefix) 교집합 개수:", len(common_prefixes))

    # 가장 많이 겹치는 prefix 10개 출력
    print("\n  공통 prefix 샘플(최대 20):")
    for p in list(common_prefixes)[:20]:
        print("   ", p)

    # 유사도 기반 매칭 (처음 30개의 JSON에 대해 가장 가까운 이미지 찾음)
    print("\n  유사도(SequenceMatcher)로 고급 매칭 시도(첫 30개 JSON 대상):")
    for j in json_norm[:30]:
        best_score = 0
        best_img = None
        for im in img_norm:
            score = SequenceMatcher(None, j, im).ratio()
            if score > best_score:
                best_score = score
                best_img = im
        if best_score > 0.9:
            print("    가능 매칭:", j, "->", best_img, f"(score={best_score:.3f})")
        # if no good match, print a short info for first few
        elif len([x for x in img_norm if x.startswith(j.split("_")[0])])>0:
            # there are images starting with same token
            pass
        else:
            # print a few non-matching json names for inspection
            if json_norm.index(j) < 10:
                print("    미매칭 샘플:", j)

# 6) unmatched 샘플 출력 (json에 있지만 폴더에 없는 샘플 몇개)
unmatched = [n for n in json_norm if n not in set_img]
print("\n미매칭 JSON 샘플 (최대 20):")
for n in unmatched[:20]:
    print("  ", n)

print("\n진단 완료 — 위 출력 결과를 붙여 주세요. (특히: JSON에서 추출한 파일명 후보 샘플 / filename-like 키 빈도 / prefix 교집합 개수 / 유사도 매칭 결과)")

🔎 JSON 파일 개수: 7

📋 JSON 내부에서 관찰된 필드(상위 몇개):
  filename: 1400
  gender: 1400
  age: 1400
  isProf: 1400
  faceExp_uploader: 1400
  bg_uploader: 1400
  annot_A: 1400
  annot_B: 1400
  annot_C: 1400

🔑 filename-like 키 빈도:
  filename: 1400

😊 emotion-like 키 빈도:
  faceExp_uploader: 1400

📦 bbox-like 키 빈도:

샘플 JSON에서 추출한 파일명 후보 (최대 20):
   '5a30dc193e2c6144184968043c00773270b111a23a3c83b00f3971c9cc8a670d_남_30_기쁨_도심 환경_20201205004857-008-002.jpg'
   '0d18ae1ed7d9e912864b736523a94a24db0595bf273964cb532e62a6087cf325_여_30_기쁨_행사&사무공간_20201207000342-006-001.jpg'
   '8f7e2f5214659c45cdcbc99a4e9ec29cf0b7571844a28e5f0de93af85aa7d76a_여_20_기쁨_문화재 및 유적지_20210129175103-009-002.jpg'
   '2964f04ec5e9bcc45276def3a6e08a3c2d1418995cf31f31d7fb9c147784c08a_남_50_기쁨_공공시설&종교&의료시설_20210125171959-001-013.jpg'
   '5feb258aabe4308f740ab36c133ef854074b138dcad3d6c66e519820dd2ec9b9_남_20_기쁨_스포츠 관람 및 레저시설_20210119120442-004-001.jpg'
   '6c95a9432ec1dc577c3d2041ead6a86d920ab836420f02ab3981b4b1f73a6d35_남_20_기쁨_상업시설&점포&시장_202

In [35]:
import os
import json
import pandas as pd

# 경로 설정
json_dir = "./json_folder"       # JSON 파일들이 들어있는 폴더
img_dir = "./image_folder"       # 이미지들이 들어있는 폴더

# 1. JSON에서 파일명 수집
json_filenames = set()
for file in os.listdir(json_dir):
    if file.endswith(".json"):
        with open(os.path.join(json_dir, file), encoding="utf-8") as f:
            data = json.load(f)
            for item in data:
                if "filename" in item:  # JSON 키 확인
                    json_filenames.add(item["filename"])

# 2. 폴더의 이미지 파일명 수집
img_filenames = set(os.listdir(img_dir))

# 3. 폴더에만 있는 파일 찾기
only_in_folder = sorted(list(img_filenames - json_filenames))

print(f"폴더에만 있는 이미지 수: {len(only_in_folder)}")
print("샘플 10개:", only_in_folder[:10])

# 4. CSV로 저장
df_only = pd.DataFrame({"filename": only_in_folder})
df_only.to_csv("only_in_folder.csv", index=False, encoding="utf-8-sig")
print("📄 only_in_folder.csv 저장 완료")

ModuleNotFoundError: No module named 'pandas'

In [31]:
import sys
print(sys.executable)

c:\ex\anaconda\envs\djangoWeb\python.exe


In [32]:
!{sys.executable} -m pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [29]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [25]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
